In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from collections import Counter
from Method.DecisionTree import DecisionTree

In [5]:
train_data = pd.read_csv("Processed Data/train.csv")
validation_data = pd.read_csv("Processed Data/validation.csv")

In [9]:
X_train = train_data.drop(columns=['Attrition_rate']).values
y_train = train_data['Attrition_rate'].values

In [22]:
model = DecisionTree(max_depth=50)
model.fit(X_train, y_train)

KeyboardInterrupt: 

In [21]:
X_val = validation_data.drop(columns=['Attrition_rate']).values
y_val = validation_data['Attrition_rate'].values

y_pred = model.predict(X_val)
rmse = mean_squared_error(y_val, y_pred, squared=False)
print(f"RMSE on validation set: {rmse}")

KeyError: 94

In [ ]:
test_data = pd.read_csv("test.csv")
X_test = test_data.drop(columns=['Attrition_rate']).values

In [ ]:
y_test = test_data['Attrition_rate'].values
y_pred_test = model.predict(X_test)

In [ ]:
rmse = mean_squared_error(y_test, y_pred_test, squared=False)
r2 = r2_score(y_test, y_pred_test)
mae = mean_absolute_error(y_test, y_pred_test)

print(f"RMSE on test set: {rmse}")
print(f"R-squared on test set: {r2}")
print(f"MAE on test set: {mae}")

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred_test, alpha=0.5)
plt.xlabel("Actual Attrition Rate")
plt.ylabel("Predicted Attrition Rate")
plt.title("Actual vs. Predicted Attrition Rate (Test Set)")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.show()

In [ ]:
errors = y_test - y_pred_test
plt.figure(figsize=(8, 6))
plt.hist(errors, bins=30) # bins: số lượng cột trong biểu đồ histogram
plt.xlabel("Prediction Errors")
plt.ylabel("Frequency")
plt.title("Distribution of Prediction Errors (Test Set)")
plt.show()


In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y_test, errors, alpha=0.5)
plt.xlabel("Actual Attrition Rate")
plt.ylabel("Prediction Error")
plt.title("Prediction Error vs. Actual Attrition Rate (Test Set)")
plt.axhline(y=0, color='k', linestyle='--', lw=2) # Đường y=0
plt.show()